# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here
orders=pd.read_csv('orders.csv')
orders.head()

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?

orders.groupby('CustomerId')

In [4]:
orders.head()

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [5]:
# your code here
test=orders.groupby(['CustomerID'])
print(test.agg({'amount_spent' : ['sum']}))
#df.quantile([.1, .25, .5, .75], axis = 0) 
test['amount_spent'].quantile(.95)

amount_spent
                    sum
CustomerID             
12346          77183.60
12347           4310.00
12348           1797.24
12349           1757.55
12350            334.40
...                 ...
18280            180.60
18281             80.82
18282            178.05
18283           2094.88
18287           1837.28

[4339 rows x 1 columns]


CustomerID
12346    77183.6000
12347       52.8150
12348      135.0000
12349       49.5000
12350       28.1600
            ...    
18280       23.6550
18281       16.9500
18282       24.0975
18283        6.2400
18287       69.6000
Name: amount_spent, Length: 4339, dtype: float64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [8]:
# your code here
test2=orders.groupby(['Country','CustomerID'])
print(test2.agg({'amount_spent' : ['sum']}))
#df.quantile([.1, .25, .5, .75], axis = 0) 
test2['amount_spent'].quantile(.95)

amount_spent
                                   sum
Country        CustomerID             
Australia      12386            401.90
               12388           2780.66
               12393           1582.60
               12415         124914.53
               12422            386.20
...                                ...
United Kingdom 18287           1837.28
Unspecified    12363            552.00
               12743            546.43
               14265            530.18
               16320           1038.46

[4347 rows x 1 columns]


Country         CustomerID
Australia       12386          73.380
                12388          62.720
                12393          49.500
                12415         488.160
                12422          33.000
                               ...   
United Kingdom  18287          69.600
Unspecified     12363          65.424
                12743          10.500
                14265          27.000
                16320          30.150
Name: amount_spent, Length: 4347, dtype: float64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [7]:
# your code here